# 7-семинар: Модельді күрделендіру, артық оқытумен күрес және сенімді бағалау

**Семинар мақсаттары:**
1.  Сызықтық емес тәуелділіктерді модельдеу үшін полиномдық регрессияны жүзеге асыру.
2.  Оқыту қисықтарының көмегімен артық оқытуды диагностикалауды үйрену.
3.  Модель сапасының робасты бағасын алу үшін кросс-валидацияны (`cross_val_score`) қолдану.
4.  Реттеуі бар модельдерді зерттеу және қолдану: `Ridge`, `Lasso`, `ElasticNet`.
5.  Реттеудің оңтайлы коэффициентін (`alpha`) автоматты түрде таңдау үшін `GridSearchCV` пайдалану.

## 1. Дайындық: деректерді жүктеп, негізгі модельді оқытамыз

Біз `Advertising.csv` деректерімен жұмысты жалғастырамыз және өткен семинарда құрған қарапайым сызықтық модельге сүйенеміз.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Деректерді жүктейміз
df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/Advertising.csv')

# X және y дайындаймыз
X = df.drop('Sales', axis=1)
y = df['Sales']

# Маңызды! Әрі қарай жұмыс істеу үшін бізге тек оқыту және тест деректері қажет болады
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 2. Полиномдық регрессия

Біздің негізгі сызықтық моделіміз жаман болған жоқ, бірақ деректердегі тәуелділіктер күрделірек болуы мүмкін. Полиномдық белгілерді (мысалы, $TV^2$, $Radio^2$, сондай-ақ олардың өзара әрекеттесуі $TV \times Radio$) қосу арқылы модельді күрделендіріп көрейік.

Ол үшін біз екі қадамды бірізді орындайтын конвейер (`pipeline`) құрамыз:
1.  `PolynomialFeatures()`: Жаңа белгілерді генерациялау.
2.  `LinearRegression()`: Осы жаңа, генерацияланған белгілерде сызықтық регрессияны оқыту.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# 2-дәрежелі полиномдық регрессия үшін конвейер құрамыз
poly_model = make_pipeline(PolynomialFeatures(degree=2, include_bias=False),
                           LinearRegression())

# Модельді оқытамыз
poly_model.fit(X_train, y_train)

### 2.1. Полиномдық модельді бағалау

Жаңа, күрделірек модельдің сапасын ескі қарапайым моделімізбен салыстырайық.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Полиномдық модельдің болжамдары
y_poly_pred = poly_model.predict(X_test)

# Метрикалар
mae_poly = mean_absolute_error(y_test, y_poly_pred)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_poly_pred))

print(f"Полиномдық модель (2-дәреже):")
print(f"MAE: {mae_poly:.2f}") # Қарапайым модельде 1.51 болған
print(f"RMSE: {rmse_poly:.2f}") # Қарапайым модельде 1.93 болған

**Қорытынды:** Модельді күрделендіру нәтиже берді! MAE және RMSE қателері едәуір азайды. Бұл деректерде шынымен де сызықтық емес тәуелділіктер (мүмкін, белгілердің өзара әрекеттесуі) болғанын және жаңа модель оларды ұстай алғанын көрсетеді.

### 2.2. Оңтайлы күрделілікті (полином дәрежесін) іздеу

Біз `degree=2` дәрежесін интуитивті түрде таңдадық. Ал егер `degree=3` немесе `degree=5` одан да жақсы болса ше? Немесе, бәлкім, біз қазірдің өзінде артық оқытып жібердік пе? Оңтайлы дәрежені табу үшін **оқыту қисықтарын** құрайық.

In [ ]:
train_rmse_errors = []
test_rmse_errors = []
degrees = range(1, 10)

for d in degrees:
    # Әр дәреже үшін модель құрып, оқытамыз
    poly_converter = PolynomialFeatures(degree=d, include_bias=False)
    X_poly_train = poly_converter.fit_transform(X_train)
    X_poly_test = poly_converter.transform(X_test) # Маңызды: fit_transform емес, transform қолданамыз
    
    model = LinearRegression()
    model.fit(X_poly_train, y_train)
    
    # Болжамдар жасап, қателерді есептейміз
    y_train_pred = model.predict(X_poly_train)
    y_test_pred = model.predict(X_poly_test)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_rmse_errors.append(train_rmse)
    test_rmse_errors.append(test_rmse)

# Оқыту қисықтарын визуализациялаймыз
plt.figure(figsize=(10, 6))
plt.plot(degrees, train_rmse_errors, label='Train RMSE')
plt.plot(degrees, test_rmse_errors, label='Test RMSE')
plt.xlabel("Полином дәрежесі")
plt.ylabel("RMSE")
plt.legend()
plt.grid()
plt.show()

**Қорытынды:** График тест іріктемесіндегі қатенің (көк сызық) 2-3 дәрежеде минималды екенін анық көрсетеді. Модельді одан әрі күрделендіру (4-дәреже және одан жоғары) тест қатесінің өсуіне әкеледі, бірақ оқытудағы қате төмендей береді. Бұл — **артық оқыту**. Демек, біз 2 немесе 3 дәрежесін таңдап дұрыс істегенбіз.

## 3. Реттеу: Артық оқытумен күрес

Енді бізде өте көп белгілер бар деп елестетейік, және жоғары дәрежелі полиномдық модель бәрібір валидацияда жақсы нәтиже көрсетіп тұр. Оны тұрақтырақ ету және артық оқытуды болдырмау үшін біз реттеуді қолдана аламыз.

**Маңызды:** Реттеу үшін деректерді **міндетті түрде** масштабтау керек! Біз `StandardScaler`-ді конвейерімізге қосамыз.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Үш қадамнан тұратын пайплайн құрайық: масштабтау, белгілерді генерациялау, модель
# Бірден оңтайлылардың бірі ретінде полиномның 3-дәрежесін алайық

# Ridge (L2)
ridge_pipe = make_pipeline(StandardScaler(),
                           PolynomialFeatures(degree=3, include_bias=False),
                           Ridge(alpha=1.0))
ridge_pipe.fit(X_train, y_train)
y_ridge_pred = ridge_pipe.predict(X_test)
print(f"Ridge үшін RMSE: {np.sqrt(mean_squared_error(y_test, y_ridge_pred)):.2f}")

# Lasso (L1)
lasso_pipe = make_pipeline(StandardScaler(),
                           PolynomialFeatures(degree=3, include_bias=False),
                           Lasso(alpha=0.01))
lasso_pipe.fit(X_train, y_train)
y_lasso_pred = lasso_pipe.predict(X_test)
print(f"Lasso үшін RMSE: {np.sqrt(mean_squared_error(y_test, y_lasso_pred)):.2f}")

### 3.1. Оңтайлы `alpha`-ны кросс-валидация көмегімен таңдау

Біз `alpha`-ны кездейсоқ таңдадық. Дұрыс тәсіл — оңтайлы мәнді кросс-валидациямен тор бойынша іздеу (`GridSearchCV`) арқылы табу.

In [ ]:
from sklearn.model_selection import GridSearchCV

# ElasticNet үшін пайплайн (L1 және L2 комбинациясы)
elastic_pipe = make_pipeline(StandardScaler(),
                             PolynomialFeatures(degree=3, include_bias=False),
                             ElasticNet(max_iter=10000))

# Тексергіміз келетін гиперпараметрлер торы
param_grid = {
    'elasticnet__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'elasticnet__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
}

# GridSearchCV нысанын құрамыз. cv=5 5-fold кросс-валидацияны білдіреді.
# scoring='neg_mean_squared_error' - оңтайландыру үшін метрика.
grid_search = GridSearchCV(estimator=elastic_pipe, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error',
                           verbose=0)

# Іздеуді іске қосамыз
grid_search.fit(X_train, y_train)

### 3.2. Іздеу нәтижелері

Қандай гиперпараметрлердің үздік болғанын көрейік.

In [ ]:
print("Үздік параметрлер:", grid_search.best_params_)

# Табылған үздік модельді тест іріктемесінде бағалайық
best_model = grid_search.best_estimator_
y_best_pred = best_model.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_best_pred))
print(f"\nТест іріктемесіндегі соңғы RMSE: {rmse_best:.2f}")

### 3.3. Реттеу эффектісін визуализациялау

Біз реттеудің коэффициенттерге қалай әсер ететінін көрдік. Ал бұл графикте қалай көрінеді? Біздің синтетикалық деректерде жоғары дәрежелі (мысалы, 15) үш полиномдық модельді оқытайық: біреуі реттеусіз, біреуі Ridge-мен және біреуі Lasso-мен. Бұл "тегістеу" эффектісін анық көруге мүмкіндік береді.

In [ ]:
# Дәрістің басында генерацияланған деректерді қолданамыз
# Мысалдың толықтығы үшін оларды осы жерде қайта анықтайық.
np.random.seed(0)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)

# Тегіс қисықтарды салу үшін деректер
X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)

# Үш модель үшін пайплайндар құрамыз. StandardScaler-ге назар аударыңыз!
degree = 15
pipe_lr = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree, include_bias=False), LinearRegression())
pipe_ridge = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree, include_bias=False), Ridge(alpha=1))
pipe_lasso = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree, include_bias=False), Lasso(alpha=0.1, max_iter=100000))

# Модельдерді оқытамыз
pipe_lr.fit(X_poly_data, y_poly_data)
pipe_ridge.fit(X_poly_data, y_poly_data)
pipe_lasso.fit(X_poly_data, y_poly_data)

# Графиктерді саламыз
plt.figure(figsize=(12, 8))
plt.scatter(X_poly_data, y_poly_data, alpha=0.4, label='Бастапқы деректер')

# Кәдімгі регрессия үшін график
y_plot_lr = pipe_lr.predict(X_plot)
plt.plot(X_plot, y_plot_lr, label='Сызықтық регрессия (артық оқыту)', color='red', linestyle='--')

# Ridge үшін график
y_plot_ridge = pipe_ridge.predict(X_plot)
plt.plot(X_plot, y_plot_ridge, label='Ridge (L2) регрессия', color='green', linewidth=3)

# Lasso үшін график
y_plot_lasso = pipe_lasso.predict(X_plot)
plt.plot(X_plot, y_plot_lasso, label='Lasso (L1) регрессия', color='purple', linewidth=3)

plt.legend()
plt.ylim(0, 10)
plt.title('Реттеуі бар және онсыз модельдерді салыстыру')
plt.grid(True)
plt.show()

**Графиктен қорытынды:**
*   **Қызыл үзік сызық (Сызықтық регрессия):** Тұрақсыз, әрбір нүктені "ұстауға" тырысып, қатты иіледі. Бұл артық оқытудың айқын мысалы.
*   **Жасыл сызық (Ridge):** Әлдеқайда тегіс және деректердегі жалпы квадраттық тенденцияны жақсырақ көрсетеді, жергілікті шығарындыларды елемейді. Ол модельді "бағындырды".
*   **Күлгін сызық (Lasso):** Сондай-ақ тегіс және робасты. Бұл жағдайда ол Ridge-ге өте ұқсас, бірақ ақпаратсыз белгілер көп басқа жағдайларда, ол одан да қарапайым модель бере алар еді (параболаға жақынырақ).

Бұл график реттеудің, үлкен коэффициенттер үшін айыппұл сала отырып, модельді қарапайым және тегіс шешімдерді табуға қалай мәжбүрлейтінін, бұл оның жалпылау қабілетін жақсартатынын анық көрсетеді.

## Қорытынды

Бұл семинарда біз алға үлкен қадам жасадық:
*   **Полиномдық регрессия** көмегімен сызықтық еместіктерді модельдеуді үйрендік.
*   Оқыту қисықтарында **артық оқыту** мәселесін көрдік және модельдің оңтайлы күрделілігін қалай таңдау керектігін түсіндік.
*   Артық оқытумен күресу және белгілерді іріктеу үшін **реттеуді** (Ridge, Lasso, ElasticNet) қолдандық.
*   Кросс-валидация көмегімен гиперпараметрлерді автоматты түрде таңдауға арналған қуатты құрал — **GridSearchCV**-ді пайдаландық.

Нәтижесінде біз күрделі, бірақ сонымен бірге робасты модель құра алдық және тест деректерінде бұрынғыдан да төмен қателікке қол жеткіздік.